# Machine Learning Price Predictor model for MTG cards

---
## 0. Untap - Project info

#### In this project, the goal is to take

#### Set restrictions:
#### <font color = "Green"> - Only non-online cards and sets will be analysed
#### - "Un-" sets will not be analysed
#### <font color = "Green"> - Sets that are comprised of masterpieces, Secret Lair Drops, Friday Night Magic Cards, Judge Gift promos and other exclusive, high-end or niche cards will not be analysed


#### - Only cards that are not part of an "Un-" set will be analysed
#### - Retail price as specified by Card Kingdom
#### - Only the price of each card at the day of its respective set or product launch will be regarded
#### - Only cards that aren't reprints will be analysed
#### <font color = "Green"> - Only cards that aren't tokens will be analysed
#### - Cards that are of the type Conspiracy, Emblem, Hero, Phenomenon, Plane, Scheme or Vanguard won't be analysed.
#### -


---
## 1. Upkeep - Importing the Libraries

In [1]:
# Data wrangling
import pandas as pd

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Machine learning model
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline

# Check work
from sklearn.utils.validation import check_is_fitted

# Supress warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

---
## 2. Draw - Importing the Data

In [2]:
# Read the MTG sets json file into a DataFrame

df = pd.read_json("data/AllPrintings.json")
print(df.shape)
df.head()

(657, 2)


,meta,data
date,2022-10-05,NaN
version,5.2.0+20221005,NaN
10E,NaN,"{'baseSetSize': 383, 'block': 'Core Set', 'boo..."
2ED,NaN,"{'baseSetSize': 302, 'block': 'Core Set', 'boo..."
2X2,NaN,"{'baseSetSize': 577, 'booster': {'default': {'..."


In [3]:
# Drop the "meta" column and the top 2 rows

df.drop(columns = ["meta"], index = ["date", "version"])

,data
10E,"{'baseSetSize': 383, 'block': 'Core Set', 'boo..."
2ED,"{'baseSetSize': 302, 'block': 'Core Set', 'boo..."
2X2,"{'baseSetSize': 577, 'booster': {'default': {'..."
2XM,"{'baseSetSize': 332, 'booster': {'default': {'..."
30A,"{'baseSetSize': 593, 'cards': [{'artist': 'Dan..."
...,...
YSNC,"{'baseSetSize': 30, 'block': 'Alchemy 2022', '..."
ZEN,"{'baseSetSize': 249, 'block': 'Zendikar', 'boo..."
ZNC,"{'baseSetSize': 1, 'block': 'Commander', 'card..."
ZNE,"{'baseSetSize': 30, 'cards': [{'artist': 'Adam..."


---
## 3. First Main Phase - Treating the data

---
#### 3a. Normalizing "df"

In [4]:
# Unpack df using json_normalize()

df_data = pd.json_normalize(df["data"])

print(df_data.shape)
df_data.head(8)

(657, 73815)


,baseSetSize,block,cards,cardsphereSetId,code,isFoilOnly,isOnlineOnly,keyruneCode,mcmId,mcmName,...,booster.default.sheets.sfcUncommon.cards.e0d350f2-2e72-583d-8123-dd28a4a32928,booster.default.sheets.sfcUncommon.cards.e10af705-d59c-5abd-afdd-c130a4ee50b3,booster.default.sheets.sfcUncommon.cards.e4d9e639-6f3d-5a34-bd14-900f31efe30b,booster.default.sheets.sfcUncommon.cards.f05ab4a8-f4ec-5406-969e-9951be880fab,booster.default.sheets.sfcUncommon.cards.f11616b0-805e-5201-843b-cc806c9ab304,booster.default.sheets.sfcUncommon.cards.f3434acf-7796-572c-a889-487c84cf7948,booster.default.sheets.sfcUncommon.cards.f4712ff2-22fe-54d1-bac8-7ef326cedb7d,booster.default.sheets.sfcUncommon.cards.f85cb2cb-34b1-5d26-bffc-eb86ebe23fbd,booster.default.sheets.sfcUncommon.cards.fdbb5212-a128-5df1-b366-c37544d24b53,booster.default.sheets.sfcUncommon.cards.ffa1ae56-535f-51de-8455-2cd52335abee
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,383.0,Core Set,"[{'artist': 'Pete Venters', 'availability': ['...",755.0,10E,False,False,10E,74.0,Tenth Edition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,302.0,Core Set,"[{'artist': 'Dan Frazier', 'availability': ['p...",938.0,2ED,False,False,2ED,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,577.0,NaN,"[{'artist': 'Mark Tedin', 'availability': ['mt...",1462.0,2X2,False,False,2X2,5070.0,Double Masters 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,332.0,NaN,"[{'artist': 'Jason Chan', 'availability': ['mt...",1251.0,2XM,False,False,2XM,3204.0,Double Masters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,593.0,NaN,"[{'artist': 'Dan Frazier', 'availability': ['p...",NaN,30A,False,False,PMEI,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,306.0,Core Set,"[{'artist': 'Dan Frazier', 'availability': ['p...",913.0,3ED,False,False,3ED,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
#### 3b. Dropping problematic columns

---
#### Not useful columns with booster and translation info

In [5]:
# Drop the top 2 rows, which contain only NaN values

df_data.drop(index = [0, 1], inplace = True)
df_data.shape

(655, 73815)

In [6]:
# Drop rows with sets that are available exclusively online

mask_online_only = df_data[df_data["isOnlineOnly"] == True]

df_data.drop(index = mask_online_only.index, inplace = True)
df_data.shape

(621, 73815)

In [7]:
# Drop columns that begin with "booster." and "translations.", which are too numerousand won't be needed

booster_columns = [col for col in df_data.columns if "booster." in col]
translations_columns = [col for col in df_data.columns if "translations." in col]

df_data.drop(columns = booster_columns, inplace = True)
df_data.drop(columns = translations_columns, inplace = True)

df_data.shape

(621, 23)

---
#### Dropping columns with too many null values

##### For the sake of methodology, "too many null values" is when a column has less than half the number of observations filled with non-null values.

In [8]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 2 to 656
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   baseSetSize       621 non-null    float64
 1   block             292 non-null    object 
 2   cards             621 non-null    object 
 3   cardsphereSetId   279 non-null    float64
 4   code              621 non-null    object 
 5   isFoilOnly        621 non-null    object 
 6   isOnlineOnly      621 non-null    object 
 7   keyruneCode       621 non-null    object 
 8   mcmId             217 non-null    float64
 9   mcmName           217 non-null    object 
 10  mtgoCode          160 non-null    object 
 11  name              621 non-null    object 
 12  releaseDate       621 non-null    object 
 13  sealedProduct     265 non-null    object 
 14  tcgplayerGroupId  294 non-null    float64
 15  tokens            621 non-null    object 
 16  totalSetSize      621 non-null    float64
 1

##### As seen above, we have some problematic columns. But how many and which ones exactly?

In [9]:
# Drop columns with too many null values

mask_too_many_null_values = [column for column in df_data.columns if df_data[column].count() < (df_data.shape[0] / 2)]
df_data.drop(columns = mask_too_many_null_values, inplace = True)

print(df_data.shape)
df_data.head()

(621, 11)


,baseSetSize,cards,code,isFoilOnly,isOnlineOnly,keyruneCode,name,releaseDate,tokens,totalSetSize,type
2,383.0,"[{'artist': 'Pete Venters', 'availability': ['...",10E,False,False,10E,Tenth Edition,2007-07-13,"[{'artist': 'Paolo Parente', 'availability': [...",508.0,core
3,302.0,"[{'artist': 'Dan Frazier', 'availability': ['p...",2ED,False,False,2ED,Unlimited Edition,1993-12-01,[],302.0,core
4,577.0,"[{'artist': 'Mark Tedin', 'availability': ['mt...",2X2,False,False,2X2,Double Masters 2022,2022-07-08,"[{'artist': 'Izzy', 'availability': ['paper'],...",579.0,masters
5,332.0,"[{'artist': 'Jason Chan', 'availability': ['mt...",2XM,False,False,2XM,Double Masters,2020-08-07,"[{'artist': 'Aleksi Briclot', 'availability': ...",384.0,masters
6,593.0,"[{'artist': 'Dan Frazier', 'availability': ['p...",30A,False,False,PMEI,30th Anniversary Edition,2022-11-28,"[{'artist': 'Jeff A. Menges', 'availability': ...",593.0,memorabilia


In [10]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 2 to 656
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   baseSetSize   621 non-null    float64
 1   cards         621 non-null    object 
 2   code          621 non-null    object 
 3   isFoilOnly    621 non-null    object 
 4   isOnlineOnly  621 non-null    object 
 5   keyruneCode   621 non-null    object 
 6   name          621 non-null    object 
 7   releaseDate   621 non-null    object 
 8   tokens        621 non-null    object 
 9   totalSetSize  621 non-null    float64
 10  type          621 non-null    object 
dtypes: float64(2), object(9)
memory usage: 58.2+ KB


---
#### Are there any duplicated sets in this DataFrame?

In [11]:
# If we get each unique "code" value in df_data, is that the same as the original DataFrame row length?

df_data["code"].unique().shape[0] == df_data.shape[0]

True

##### <font color = "Green"> There aren't any duplicated sets in the DataFrame

---
#### Analysing specific columns and pondering their utility

#### isFoilOnly column

In [12]:
df_data["isFoilOnly"].value_counts()

False    435
True     186
Name: isFoilOnly, dtype: int64

##### As we can see, there are some rows which are comprised solely of foil cards. But what sets are those?

In [13]:
mask_foil_only = df_data[df_data["isFoilOnly"] == True]
mask_foil_only["name"]

73              Commander's Arsenal
83            Magic 2015 Clash Pack
84         Fate Reforged Clash Pack
85         Magic Origins Clash Pack
119         From the Vault: Dragons
                   ...             
626    From the Vault: Annihilation
627          From the Vault: Angels
628            From the Vault: Lore
629       From the Vault: Transform
645      World Magic Cup Qualifiers
Name: name, Length: 186, dtype: object

##### Looking at the list above, we can see that the all-foil rows are comprised of premium mtg products like Judge Gift Cards, Oversized Commander Cards,
##### Foil Secret Lair Drops, Masterpieces ("Zendikar Expeditions", "Kaladesh Inventions"...) etc. For the sake of methodology, they will all be disregarded.

#### "type" column

In [29]:
df_data["type"].unique()

array(['core', 'masters', 'memorabilia', 'commander', 'expansion',
       'archenemy', 'box', 'draft_innovation', 'masterpiece', 'arsenal',
       'funny', 'duel_deck', 'promo', 'planechase', 'starter', 'token',
       'vanguard', 'spellbook'], dtype=object)

##### Thanks to the "type" column, we can see that there are still sets that need to be removed

In [56]:
# Drop sets considered "memorabilia", collectible, funny, masterpiece etc.

type_col = df_data["type"]

mask_archenemy_schemes = (type_col == "archenemy") & (df_data["name"].str.contains("Schemes")) # Select sets only with scheme cards
mask_planechase_planes = (type_col == "planechase") & (df_data["name"].str.contains("Planes")) # Select sets only with plane cards
mask_memorabilia = type_col == "memorabilia"
mask_masterpiece = type_col == "masterpiece"
mask_secret_lair = df_data["name"].str.contains("Secret Lair")
mask_arsenal = type_col == "arsenal" # Select Commander Collection products (currently only Green and Black)
mask_unsets = type_col == "funny"
mask_promo = type_col == "promo"
mask_token = type_col == "token"
mask_vanguard = type_col == "vanguard"
mask_spellbook = type_col == "spellbook"

(df_data.drop(
    index = df_data[
        mask_archenemy_schemes |
        mask_planechase_planes |
        mask_memorabilia |
        mask_masterpiece |
        mask_secret_lair |
        mask_arsenal |
        mask_unsets |
        mask_promo |
        mask_token |
        mask_vanguard |
        mask_spellbook
        ].index, inplace = True)
)

In [58]:
df_data.shape

(226, 8)

In [14]:
# Drop rows that contain all-foil sets

print(f"shape before = {df_data.shape}")

df_data.drop(index = mask_foil_only.index, inplace = True)

print(f"shape after = {df_data.shape}")

shape before = (621, 11)
shape after = (435, 11)


#### isFoilOnly and isOnlineOnly columns

In [15]:
df_data["isFoilOnly"].value_counts()

False    435
Name: isFoilOnly, dtype: int64

In [16]:
df_data["isOnlineOnly"].value_counts()

False    435
Name: isOnlineOnly, dtype: int64

##### Due to our data treatment, the "isOnlineOnly" and "isFoilOnly" columns are unanimous in their values.
##### Therefore, they are no longer useful for our wrangling and analysis and can be safely discarded.

In [17]:
# Drop "isOnlineOnly" and "isFoilOnly" columns

print(f"shape before = {df_data.shape}")

df_data.drop(columns = ["isOnlineOnly", "isFoilOnly"], inplace = True)

print(f"shape after = {df_data.shape}")

shape before = (435, 11)
shape after = (435, 9)


#### "tokens" column

##### Since we are not going to analyse tokens, the "tokens" column can be safely discarded

In [18]:
print(f"shape before = {df_data.shape}")

df_data.drop(columns = ["tokens"], inplace = True)

print(f"shape after = {df_data.shape}")

shape before = (435, 9)
shape after = (435, 8)


#### "baseSetSize" or "totalSetSize"?

##### According to MTGJSON, "baseSetSize" and "totalSetSize" are defined as follows:
##### - baseSetSize: The number of cards in the set. Wizards of the Coast sometimes prints extra cards beyond the set size into promos or supplemental products.
##### - totalSetSize: The total number of cards in the set, including promotional and related supplemental products but excluding Alchemy modifications.

##### <font color = "Green"> Since we don't yet know the impact of a set size on a card's price, I have decided to keep both columns for further analysis

In [19]:
df_data.head()

,baseSetSize,cards,code,keyruneCode,name,releaseDate,totalSetSize,type
2,383.0,"[{'artist': 'Pete Venters', 'availability': ['...",10E,10E,Tenth Edition,2007-07-13,508.0,core
3,302.0,"[{'artist': 'Dan Frazier', 'availability': ['p...",2ED,2ED,Unlimited Edition,1993-12-01,302.0,core
4,577.0,"[{'artist': 'Mark Tedin', 'availability': ['mt...",2X2,2X2,Double Masters 2022,2022-07-08,579.0,masters
5,332.0,"[{'artist': 'Jason Chan', 'availability': ['mt...",2XM,2XM,Double Masters,2020-08-07,384.0,masters
6,593.0,"[{'artist': 'Dan Frazier', 'availability': ['p...",30A,PMEI,30th Anniversary Edition,2022-11-28,593.0,memorabilia


---
#### 3c. Categorical and Numerical variables and their types.

In [20]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435 entries, 2 to 656
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   baseSetSize   435 non-null    float64
 1   cards         435 non-null    object 
 2   code          435 non-null    object 
 3   keyruneCode   435 non-null    object 
 4   name          435 non-null    object 
 5   releaseDate   435 non-null    object 
 6   totalSetSize  435 non-null    float64
 7   type          435 non-null    object 
dtypes: float64(2), object(6)
memory usage: 30.6+ KB


In [21]:
for column in df_data.columns:
    print(f"{df_data[column].name}         type:         {type(df_data[column].iloc[0])}")

baseSetSize         type:         <class 'numpy.float64'>
cards         type:         <class 'list'>
code         type:         <class 'str'>
keyruneCode         type:         <class 'str'>
name         type:         <class 'str'>
releaseDate         type:         <class 'str'>
totalSetSize         type:         <class 'numpy.float64'>
type         type:         <class 'str'>


##### Upon a closer look at the variable types, one column sticks out: "releaseDate", currently filled with string values.
##### All values in the column will be changed to "datetime"

In [22]:
df_data["releaseDate"] = pd.to_datetime(df_data["releaseDate"], yearfirst = True, format = "%Y-%m-%d")

In [23]:
type(df_data["releaseDate"].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [26]:
df_data.head(10)

,baseSetSize,cards,code,keyruneCode,name,releaseDate,totalSetSize,type
2,383.0,"[{'artist': 'Pete Venters', 'availability': ['...",10E,10E,Tenth Edition,2007-07-13,508.0,core
3,302.0,"[{'artist': 'Dan Frazier', 'availability': ['p...",2ED,2ED,Unlimited Edition,1993-12-01,302.0,core
4,577.0,"[{'artist': 'Mark Tedin', 'availability': ['mt...",2X2,2X2,Double Masters 2022,2022-07-08,579.0,masters
5,332.0,"[{'artist': 'Jason Chan', 'availability': ['mt...",2XM,2XM,Double Masters,2020-08-07,384.0,masters
6,593.0,"[{'artist': 'Dan Frazier', 'availability': ['p...",30A,PMEI,30th Anniversary Edition,2022-11-28,593.0,memorabilia
7,306.0,"[{'artist': 'Dan Frazier', 'availability': ['p...",3ED,3ED,Revised Edition,1994-04-01,306.0,core
8,321.0,"[{'artist': 'Anton Solovianchyk', 'availabilit...",40K,40K,"Warhammer 40,000",2022-10-07,321.0,commander
9,378.0,"[{'artist': 'Harold McNeill', 'availability': ...",4BB,4ED,Fourth Edition Foreign Black Border,1995-04-01,378.0,core
10,378.0,"[{'artist': 'Harold McNeill', 'availability': ...",4ED,4ED,Fourth Edition,1995-04-01,379.0,core
11,165.0,"[{'artist': 'John Matson', 'availability': ['m...",5DN,5DN,Fifth Dawn,2004-06-04,165.0,expansion


#### <font color = "blue"> From what it looks like, the information about the cards themselves are in the "cards" column, whose value type is list.
#### <font color = "blue"> These lists contain dictionaries, each one referencing an MTG card.

In [24]:
# type(df_data["cards"].iloc[2])

###

In [25]:
#df_data_cards = [
#    df_data.from_dict(df_data["cards"])
#    .iloc[mtg_set]
#    for mtg_set in df_data["cards"].index
#]

#print(len(df_data_cards))